# Generador de Datasets Sinteticos con LLMs

Este notebook permite generar datasets estructurados utilizando modelos de lenguaje.

**Caracteristicas:**
- Define un esquema (campos y tipos de datos)
- Especifica el contexto/dominio de los datos
- Genera automaticamente la cantidad de registros que necesites
- Parsea, valida y deduplica los resultados

**Modelos soportados:**
- OpenAI (GPT-4o, GPT-4o-mini)
- Anthropic (Claude 3.5 Sonnet, Haiku)
- Google (Gemini 2.0, 1.5)
- Hugging Face (Llama, Gemma, Mistral, Qwen)

## 1. Instalacion de Dependencias

In [1]:
!pip install -q transformers accelerate bitsandbytes
!pip install -q openai anthropic google-generativeai
!pip install -q datasets pandas tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 26.7 MB/s eta 0:00:00


In [16]:
import os
import re
import json
import hashlib
import pandas as pd
import time
from tqdm.auto import tqdm
from typing import List, Dict, Optional
from google.colab import userdata

## 2. Configuracion de APIs y Modelos

In [17]:
try:
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    HF_TOKEN = userdata.get('HF_TOKEN')
    print("Claves cargadas desde Colab Secrets")
except Exception as e:
    print(e)

print(f"OpenAI: {'OK' if OPENAI_API_KEY else 'No configurada'}")
print(f"Anthropic: {'OK' if ANTHROPIC_API_KEY else 'No configurada'}")
print(f"Google: {'OK' if GOOGLE_API_KEY else 'No configurada'}")
print(f"HuggingFace: {'OK' if HF_TOKEN else 'No configurado'}")

SYSTEM_MESSAGE = "Eres un generador de datos sinteticos"

OPENAI_MODEL="gpt-4o-mini"
ANTHROPIC_MODEL="claude-sonnet-4-5-20250929"
GOOGLE_MODEL="gemini-2.0-flash"
HF_MODEL="meta-llama/Llama-3.2-3B-Instruct"

Claves cargadas desde Colab Secrets
OpenAI: OK
Anthropic: OK
Google: OK
HuggingFace: OK


## 3. Clases de Generadores

In [14]:
class OpenAIGenerator:
    def __init__(self, model: str = OPENAI_MODEL):
        from openai import OpenAI
        self.client = OpenAI()
        self.model = model

    def generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.7) -> str:
        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": SYSTEM_MESSAGE},
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature
        )
        return response.choices[0].message.content


class AnthropicGenerator:
    def __init__(self, model: str = ANTHROPIC_MODEL):
        from anthropic import Anthropic
        self.client = Anthropic()
        self.model = model

    def generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.7) -> str:
        response = self.client.messages.create(
            model=self.model,
            max_tokens=max_tokens,
            temperature=temperature,
            system=SYSTEM_MESSAGE,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response.content[0].text


class GeminiGenerator:
    def __init__(self, model: str = GOOGLE_MODEL):
        import google.generativeai as genai
        genai.configure()
        self.model = genai.GenerativeModel(
            model_name=model,
            system_instruction=SYSTEM_MESSAGE
        )

    def generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.7) -> str:
        response = self.model.generate_content(
            prompt,
            generation_config={
                'temperature': temperature,
                'max_output_tokens': max_tokens,
            }
        )
        return response.text


class HuggingFaceGenerator:
    def __init__(self, model_name: str = HF_MODEL):
        from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer
        import torch
        self.model_name = model_name
        print(f"Cargando modelo: {model_name}")
        quant_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4"
        )
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            quantization_config=quant_config,
            device_map="auto"
        )
        print("Modelo cargado")

    def generate(self, prompt: str, max_tokens: int = 512, temperature: float = 0.7) -> str:
        messages = [
            {"role": "system", "content": SYSTEM_MESSAGE},
            {"role": "user", "content": prompt}
        ]
        inputs = self.tokenizer.apply_chat_template(
            messages,
            return_tensors="pt"
        ).to("cuda")

        streamer = TextStreamer(tokenizer)

        outputs = self.model.generate(
            inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            pad_token_id=self.tokenizer.eos_token_id,
            streamer=streamer
        )
        response = self.tokenizer.decode(
            outputs[0][inputs.shape[1]:],
            skip_special_tokens=True
        )
        return response.strip()

## 4. Generador de Datasets Sinteticos

In [15]:
class SyntheticDatasetGenerator:
    """Generador de datasets sinteticos estructurados usando LLMs"""

    def __init__(self, generator):
        self.generator = generator
        self.generated_data = []
        # self.seen_hashes = set()

    def _build_prompt(self, schema: Dict, context: str, batch_size: int, examples: List[Dict] = None) -> str:
        schema_str = json.dumps(schema, indent=2, ensure_ascii=False)

        examples_str = ""
        if examples:
            examples_str = f"\n\nEjemplos de referencia (genera datos DIFERENTES a estos):\n{json.dumps(examples[:3], indent=2, ensure_ascii=False)}"

        prompt = f"""Genera exactamente {batch_size} registros de datos sinteticos realistas.

CONTEXTO: {context}

ESQUEMA (cada registro debe tener exactamente estos campos):
{schema_str}
{examples_str}

REGLAS:
1. Genera datos realistas y variados
2. Respeta los tipos de datos indicados
3. NO repitas datos, cada registro debe ser unico
4. Responde UNICAMENTE con un array JSON valido, sin explicaciones
5. NO uses markdown, solo el JSON puro

Respuesta (array JSON de {batch_size} objetos):"""

        return prompt

#    def _parse_response(self, response: str) -> List[Dict]:
#        response = response.strip()
#
#        json_match = re.search(r'\[[\s\S]*\]', response)
#        if json_match:
#            try:
#                return json.loads(json_match.group())
#            except json.JSONDecodeError:
#                pass
#
#        try:
#            data = json.loads(response)
#            if isinstance(data, list):
#                return data
#            elif isinstance(data, dict):
#                return [data]
#        except json.JSONDecodeError:
#            pass
#
#        print("  [WARN] No se pudo parsear la respuesta como JSON")
#        return []
#
#    def _validate_record(self, record: Dict, schema: Dict) -> bool:
#        for field in schema.keys():
#            if field not in record:
#                return False
#            if record[field] is None or record[field] == "":
#                return False
#        return True
#
#    def _get_hash(self, record: Dict) -> str:
#        record_str = json.dumps(record, sort_keys=True, ensure_ascii=False)
#        return hashlib.md5(record_str.encode()).hexdigest()
#
#    def _deduplicate(self, records: List[Dict]) -> List[Dict]:
#        unique_records = []
#        for record in records:
#            record_hash = self._get_hash(record)
#            if record_hash not in self.seen_hashes:
#                self.seen_hashes.add(record_hash)
#                unique_records.append(record)
#        return unique_records

    def generate(
        self,
        schema: Dict[str, str],
        context: str,
        num_records: int,
        batch_size: int = 20,
        max_retries: int = 3,
        temperature: float = 0.8
    ) -> pd.DataFrame:
        """
        Genera un dataset sintetico estructurado.

        Args:
            schema: Diccionario {campo: descripcion_tipo}
            context: Descripcion del dominio/contexto de los datos
            num_records: Cantidad total de registros a generar
            batch_size: Registros por llamada al LLM (10-30 recomendado)
            max_retries: Reintentos si falla una generacion
            temperature: Creatividad (0.5-1.0 recomendado)

        Returns:
            DataFrame con los datos generados
        """
        self.generated_data = []
        self.seen_hashes = set()

        # pbar = tqdm(total=num_records, desc="Generando")

        while len(self.generated_data) < num_records:
            remaining = num_records - len(self.generated_data)
            current_batch = min(batch_size, remaining)

            examples = self.generated_data[-5:] if self.generated_data else None
            prompt = self._build_prompt(schema, context, current_batch, examples)

            for attempt in range(max_retries):
                try:
                    response = self.generator.generate(
                        prompt=prompt,
                        max_tokens=2000,
                        temperature=temperature
                    )

                    records = self._parse_response(response)
                    valid_records = [r for r in records if self._validate_record(r, schema)]
                    # unique_records = self._deduplicate(valid_records)

                    # if unique_records:
                    if valid_records:
                        # self.generated_data.extend(unique_records)
                        # pbar.update(len(unique_records))
                        # break
                    else:
                        print(f"  [WARN] Intento {attempt + 1}: Sin registros validos")

                except Exception as e:
                    print(f"  [ERROR] Intento {attempt + 1}: {e}")
                    if attempt == max_retries - 1:
                        print("  Continuando con los datos obtenidos...")

                time.sleep(1)

            time.sleep(0.5)

        # pbar.close()
        self.generated_data = self.generated_data[:num_records]

        df = pd.DataFrame(self.generated_data)
        print(f"\nDataset generado: {len(df)} registros unicos")
        return df

    def save(self, filename: str, format: str = 'csv'):
        """Guarda el dataset generado"""
        df = pd.DataFrame(self.generated_data)

        if format == 'csv':
            df.to_csv(filename, index=False)
        elif format == 'json':
            df.to_json(filename, orient='records', indent=2, force_ascii=False)
        elif format == 'jsonl':
            df.to_json(filename, orient='records', lines=True, force_ascii=False)

        print(f"Guardado en: {filename}")
        return filename

## 5. Ejemplos de Uso

### Crear generador

In [9]:
# Elige el generador segun la API que tengas configurada:

# generator = OpenAIGenerator("gpt-4o-mini")
# generator = AnthropicGenerator("claude-3-5-sonnet-20241022")
# generator = GeminiGenerator("gemini-2.0-flash-exp")
# generator = HuggingFaceGenerator("meta-llama/Llama-3.2-3B-Instruct")

generator = OpenAIGenerator("gpt-4o-mini")
synth_gen = SyntheticDatasetGenerator(generator)

### Ejemplo 1: Dataset de Ventas

In [10]:
schema_ventas = {
    "producto": "nombre del producto (electronica, ropa, hogar)",
    "categoria": "categoria del producto",
    "precio": "precio en euros (numero decimal)",
    "cantidad_vendida": "unidades vendidas (entero)",
    "fecha_venta": "fecha en formato YYYY-MM-DD",
    "ciudad": "ciudad de Espana donde se realizo la venta"
}

df_ventas = synth_gen.generate(
    schema=schema_ventas,
    context="Ventas de una tienda online de electronica en Espana durante 2024",
    num_records=10,
    batch_size=5,
    temperature=0.8
)

display(df_ventas.head(10))

Generando:   0%|          | 0/100 [00:00<?, ?it/s]


Dataset generado: 100 registros unicos


,producto,categoria,precio,cantidad_vendida,fecha_venta,ciudad
0,Smartphone XPro,electronica,799.99,15,2024-01-15,Madrid
1,Auriculares Wireless Z3,electronica,149.95,50,2024-01-20,Barcelona
2,Laptop SlimBook 14,electronica,999.00,10,2024-02-02,Valencia
3,Televisor 55' Ultra HD,electronica,599.99,8,2024-02-12,Sevilla
4,Smartwatch Fitness Pro,electronica,249.99,30,2024-02-20,Bilbao
5,Cámara DSLR 24MP,electronica,749.99,5,2024-03-01,Malaga
6,Altavoz Bluetooth Mini,electronica,99.99,45,2024-03-10,Zaragoza
7,Tablet 10 pulgadas,electronica,299.99,20,2024-03-15,Granada
8,Consola de Videojuegos X,electronica,399.99,25,2024-03-25,Alicante
9,Router WiFi 6,electronica,129.99,40,2024-04-05,Tarragona


### Ejemplo 2: Dataset de Paises

In [12]:
schema_paises = {
    "pais": "nombre del pais",
    "capital": "nombre de la capital",
    "continente": "continente donde se encuentra",
    "poblacion_millones": "poblacion aproximada en millones (numero)",
    "idioma_oficial": "idioma oficial principal",
    "moneda": "nombre de la moneda oficial"
}

df_paises = synth_gen.generate(
    schema=schema_paises,
    context="Paises del mundo con datos geograficos y demograficos",
    num_records=10,
    batch_size=5,
    temperature=0.5
)

display(df_paises.head(10))

Generando:   0%|          | 0/15 [00:00<?, ?it/s]

  [WARN] Intento 1: Sin registros validos
  [WARN] Intento 2: Sin registros validos
  [WARN] Intento 3: Sin registros validos
  [WARN] Intento 1: Sin registros validos
  [WARN] Intento 2: Sin registros validos
  [WARN] Intento 3: Sin registros validos
  [WARN] Intento 1: Sin registros validos
  [WARN] Intento 2: Sin registros validos
  [WARN] Intento 3: Sin registros validos
  [WARN] Intento 1: Sin registros validos
  [WARN] Intento 2: Sin registros validos

Dataset generado: 15 registros unicos


,pais,capital,continente,poblacion_millones,idioma_oficial,moneda
0,Argentina,Buenos Aires,América del Sur,45,Español,Peso argentino
1,Japón,Tokio,Asia,126,Japonés,Yen japonés
2,Alemania,Berlín,Europa,83,Alemán,Euro
3,Sudáfrica,Pretoria,África,59,"Afrikáans, Inglés, entre otros",Rand sudafricano
4,Australia,Canberra,Oceanía,25,Inglés,Dólar australiano
5,Canadá,Ottawa,América del Norte,38,Inglés y Francés,Dólar canadiense
6,Brasil,Brasilia,América del Sur,213,Portugués,Real brasileño
7,India,Nueva Delhi,Asia,1390,Hindi y Inglés,Rupia india
8,Francia,París,Europa,65,Francés,Euro
9,Egipto,El Cairo,África,104,Árabe,Libra egipcia


### Guardar Datasets

In [ ]:
# Guardar en diferentes formatos
synth_gen.save('dataset_ventas.csv', format='csv')
synth_gen.save('dataset_ventas.json', format='json')

# O directamente desde el DataFrame
df_paises.to_csv('dataset_paises.csv', index=False)

print("Datasets guardados")